In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import numpy as np
import gzip
import io
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [3]:
with gzip.open(io.BytesIO(req.get('https://content.novayagazeta.ru/sitemap.xml.gz').content)) as f:
    sitemap_urls = parse_sitemap(io.StringIO(f.read().decode()))

sitemap_urls[:5]

[{'url': 'https://content.novayagazeta.ru/sitemap1.xml.gz',
  'lastmod': '2018-07-31T05:00:42+03:00'},
 {'url': 'https://content.novayagazeta.ru/sitemap2.xml.gz',
  'lastmod': '2018-07-31T05:00:43+03:00'},
 {'url': 'https://content.novayagazeta.ru/sitemap3.xml.gz',
  'lastmod': '2018-07-31T05:01:04+03:00'},
 {'url': 'https://content.novayagazeta.ru/sitemap4.xml.gz',
  'lastmod': '2018-07-31T05:01:33+03:00'},
 {'url': 'https://content.novayagazeta.ru/sitemap5.xml.gz',
  'lastmod': '2018-07-31T05:01:50+03:00'}]

In [4]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemap_urls):
    try:
        with gzip.open(io.BytesIO(req.get(fn['url']).content)) as f:
            sitemap = io.StringIO(f.read().decode())
        news = parse_sitemap(sitemap)

    except Exception as e:
        print(e)
        continue

    news_db = news_db + news

<ipython-input-4-eead7f749a4c>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemap_urls):


In [11]:
df = pd.DataFrame(news_db).iloc[1:]
df['lastmod'] = df['url'].str.extract('(\d+\/\d+\/\d+)')
df = df.dropna()
df['lastmod'] = pd.to_datetime(df['lastmod'])
df.head()

,url,lastmod
1,https://www.novayagazeta.ru/news/2010/01/02/1-...,2010-01-02
2,https://www.novayagazeta.ru/news/2010/01/02/2-...,2010-01-02
3,https://www.novayagazeta.ru/news/2010/01/02/3-...,2010-01-02
4,https://www.novayagazeta.ru/news/2010/01/04/4-...,2010-01-04
5,https://www.novayagazeta.ru/news/2010/01/04/5-...,2010-01-04


In [13]:
df[(df.lastmod >= '2019-11-01')].to_json('news_novaya.json', date_unit="s", date_format="iso")
df[(df.lastmod >= '2019-11-01')].shape

(9726, 2)

# Parsing

In [19]:
def parse_news(text):
    soup = BeautifulSoup(text)
    news_page = soup.find(name="div", attrs={'id': "selection-range-available"})

    if news_page:
        news_text = ''
        news_elems = news_page.find_all(name=['p', 'dl'])
        for news_elem in news_elems:
            news_text += news_elem.get_text(" ", strip=True)
        news_text = re.sub(r'[\n\r]+', ' ', news_text).replace(u'\xa0', u' ')
    else:
        news_text = ''

    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+', ' ', description_elem.get('content')).replace(u'\xa0', u' ') if description_elem else ''
    
    tags_elem = soup.find(name="meta", attrs={'name': "keywords"})
    tags = re.sub(r'[\n\r]+', ' ', tags_elem.get('content')).replace(u'\xa0', u' ') if tags_elem else ''
    return news_text, tags, description

In [20]:
parse_news(req.get('https://www.novayagazeta.ru/news/2018/01/02/138378-bitkoin-startoval-s-padeniya-vpervye-za-neskolko-let').text)

('Впервые с 2015 года курс биткоина начал новый год со снижения, подсчитал Bloomberg. Сейчас он составляет 13 167 долларов, свидетельствуют данные сервиса Coindesk.В первый день 2018 года биткоин, по данным сайта, упал на 4,28%, а в 2015 году криптовалюта потеряла 1,855% стоимости.В первый день прошлого года криптовалюта выросла на 3,58%. Затем биткоин сохранил рост, чем вызвал мировую лихорадку вокруг криптовалют, отмечает Bloomberg.В конце марта 2017 года биткоин стоил около тысячи долларов. Валюта стала стремительно расти в начале апреля, самый значительный рост пришелся на вторую половину ноября.В середине декабря курс биткоина впервые превысил порог в 20 тысяч долларов, однако потом резко упал на 15%.22 декабря криптовалюта потеряла 20% стоимости за сутки: курс биткоина на бирже опустился ниже 14 тысяч долларов. За неделю биткоин потерял в цене около шести тысяч долларов. Bloomberg отмечал, что резкое снижение курса может быть связано с его коррекцией после масштабных денежных вли

In [10]:
for i, item in enumerate(df.iterrows()):

    if not df.loc[item[0], 'downloaded']:
        
        try:
            # Processing row from db
            url = item[1]['url']
            lastmod = item[1]['lastmod']
            csv_row = dict(zip(['news_id', 'url', 'lastmod'], [item[0], url, lastmod]))

            # Downloading file
            start = time.time()
            resp = req.get(url)
            if resp.status_code == 200:
                csv_row['news_text'], csv_row['tags'], csv_row['description'] = parse_rbk_news(resp.text)

            # Saving row to csv file
            with open('db/news_rbk_part.csv', 'a') as csv_file:
                writer = csv.DictWriter(csv_file, csv_row.keys())
                writer.writerow(csv_row)

            # News is written, logging it to DF
            #df.loc[item[0], 'downloaded'] = True
            
            # Logging to console
            if i % 1000 == 0 and i > 0:
                #df.to_json('db/news_rbk_part.json')
                pass
                
            if i % 100 == 0 and i > 0:
                datetime = time.strftime('%Y-%m-%d %H:%M')
                print(datetime, ':', i, 'news downloaded,', int((df.shape[0]-i) * 1.1 / 60 / 60), 'hours remaining')

            # Calculating wait time (must be 1 sec minimum)
            end = time.time()
            wait = max(0, 1.1 - (end - start))
            time.sleep(wait)
            break
        except Exception as e:
            #df.to_json('db/news_rbk_part.json')
            print(e)
            break